# Import packages and database

In [1]:
import pandas as pd
import numpy as np
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.segmentation as seg
from biogeme.expressions import Beta, Variable, Derive, exp
from biogeme import models
from biogeme import results as res
from biogeme.expressions import DefineVariable, log
from collections import namedtuple

data_file ='https://raw.githubusercontent.com/GustavePellier/MMOB/main/lpmc19.dat'
LPMC = pd.read_csv(data_file, sep='\t')
LPMC

database = db.Database('LPMC', LPMC)
all_results = {}

# Model 0

We calculate the total public transport duration and the total driving cost

In [2]:
LPMC["dur_pt"]= LPMC["dur_pt_access"] + LPMC["dur_pt_rail"] + LPMC["dur_pt_bus"] + LPMC["dur_pt_int"] 
LPMC["cost_drive"] = LPMC["cost_driving_ccharge"] + LPMC["cost_driving_fuel"]

Some variables are created with the columns that seem to be useful

In [3]:
travel_mode=Variable('travel_mode')

dur_pt_access=Variable('dur_pt_access')
dur_pt_rail=Variable('dur_pt_rail')
dur_pt_bus=Variable('dur_pt_bus')
dur_pt_int=Variable('dur_pt_int')

pt_interchanges=Variable('pt_interchanges')

cost_driving_fuel=Variable('cost_driving_fuel')
cost_driving_ccharge=Variable('cost_driving_ccharge')
cost_drive=Variable('cost_drive')
cost_pt=Variable('cost_transit')

time_walk=Variable('dur_walking')
time_cycle=Variable('dur_cycling')
time_pt=Variable('dur_pt')
time_drive=Variable('dur_driving')

female=Variable('female')
age=Variable('age')

There are 4 different travel mode, we will thus build a model with 4 utility functions, we create 3 alternative specific constant, a generic parameter for travel time and a generic parameter for cost

In [4]:
asc_cycle = Beta('asc_cycle', 0, None, None, 0)
asc_pt = Beta('asc_pt', 0, None, None, 0)
asc_drive = Beta('asc_drive', 0, None, None, 0)
beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_time = Beta('beta_time', 0, None, None, 0)

Utility functions creation

In [5]:
v_walk_model0= beta_time * time_walk  
v_cycle_model0= asc_cycle + beta_time * time_cycle 
v_pt_model0= asc_pt + beta_time * time_pt + beta_cost * cost_pt
v_drive_model0= asc_drive + beta_time * time_drive + beta_cost * cost_drive

availability of each mode, all available here

In [6]:
av = {1: 1, 2: 1, 3: 1, 4:1}

The estimation results (parameter values, t-tests or p-values, null and final log likelihoods)

In [7]:
V_model0 = {1: v_walk_model0 , 2: v_cycle_model0, 3: v_pt_model0, 4: v_drive_model0}
logprob_model0 = models.loglogit(V_model0, av, travel_mode)
biogeme_model0 = bio.BIOGEME(database, logprob_model0)
biogeme_model0.modelName = 'Model_0'
all_results['Model0'] = biogeme_model0.estimate()
results_generic = biogeme_model0.estimate()
results_generic.getEstimatedParameters()

File biogeme.toml has been created


,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-3.878619,0.107672,-36.022418,0.0
asc_drive,-1.295191,0.080505,-16.088354,0.0
asc_pt,-0.503354,0.054123,-9.300119,0.0
beta_cost,-0.193629,0.013958,-13.871889,0.0
beta_time,-5.495527,0.208596,-26.345342,0.0


In [8]:
res.compile_estimation_results(all_results)

AttributeError: module 'biogeme.results' has no attribute 'compile_estimation_results'

In [9]:
print("Null Loglikelihood : ")
biogeme_model0.calculateNullLoglikelihood(av)

Null Loglikelihood : 


-6931.471805599917

# Model 1

We choose to do a specification for the time attribute, the cost stays generic in this model. Therefore we obtain a beta_time parameter for each transportation mode. 

In [10]:
beta_time_walk = Beta('beta_time_walk', 0, None, None, 0)
beta_time_cycle = Beta('beta_time_cycle', 0, None, None, 0)
beta_time_pt = Beta('beta_time_pt', 0, None, None, 0)
beta_time_drive = Beta('beta_time_drive', 0, None, None, 0)

The utility functions are rewritten with the new parameters :

In [11]:
v_walk_model1= beta_time_walk * time_walk  
v_cycle_model1= asc_cycle + beta_time_cycle * time_cycle 
v_pt_model1= asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt
v_drive_model1= asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive

The results are computed (parameter values, t-tests or p-values, null and final log likelihoods)

In [12]:
V_model1 = {1: v_walk_model1 , 2: v_cycle_model1, 3: v_pt_model1, 4: v_drive_model1}
logprob_model1 = models.loglogit(V_model1, av, travel_mode)
biogeme_model1 = bio.BIOGEME(database, logprob_model1)
biogeme_model1.modelName = 'Model_1'
all_results['Model1'] = biogeme_model1.estimate()
results_specific_time = biogeme_model1.estimate()
results_specific_time.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-4.847427,0.202567,-23.929963,0.0
asc_drive,-1.969868,0.137928,-14.281882,0.0
asc_pt,-2.337230,0.139505,-16.753691,0.0
beta_cost,-0.182006,0.016086,-11.314629,0.0
beta_time_cycle,-5.311244,0.461876,-11.499292,0.0
beta_time_drive,-6.473579,0.378946,-17.083101,0.0
beta_time_pt,-3.538116,0.250631,-14.116823,0.0
beta_time_walk,-8.430537,0.418984,-20.121406,0.0


In [13]:
res.compile_estimation_results(all_results)

AttributeError: module 'biogeme.results' has no attribute 'compile_estimation_results'

In [ ]:
biogeme_model1.calculateNullLoglikelihood(av)

We compare the 2 models with the likelihood ratio test because model 0 is a reduction of model 1.  

In [ ]:
results_specific_time.likelihood_ratio_test(results_generic, 0.05)

# Model 2

Using Model 1 as the base model, we choose to study how gender interact with the
ASCs and the alternative attributes. 

Lets start by segmenting the population by gender

In [14]:
gender_segmentation = seg.DiscreteSegmentationTuple(variable=female, mapping={1: 'female', 0: 'male'})

### ASC segmentation

In [15]:
asc_cycle = Beta('asc_cycle', 0, None, None, 0)
segmented_asc_cycle = seg.Segmentation(asc_cycle,[gender_segmentation]).segmented_beta()

asc_pt = Beta('asc_pt', 0, None, None, 0)
segmented_asc_pt = seg.Segmentation(asc_pt,[gender_segmentation]).segmented_beta()

asc_drive = Beta('asc_drive', 0, None, None, 0)
segmented_asc_drive = seg.Segmentation(asc_drive,[gender_segmentation]).segmented_beta()

We redefine the value function of the 4 alternatives, introducing the segmented ASC and the segmented cost attribute

In [16]:
v_walk_model2= beta_time_walk * time_walk  
v_cycle_model2= segmented_asc_cycle + beta_time_cycle * time_cycle 
v_pt_model2= segmented_asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt
v_drive_model2= segmented_asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive

We can compute the result of the new model

In [17]:
V_model2 = {1: v_walk_model2 , 2: v_cycle_model2, 3: v_pt_model2, 4: v_drive_model2}
logprob_model2 = models.loglogit(V_model2, av, travel_mode)
biogeme_model2 = bio.BIOGEME(database, logprob_model2)
biogeme_model2.modelName = 'Model_2'
all_results['Model2'] = biogeme_model2.estimate()
results_segmented_gender = biogeme_model2.estimate()
results_segmented_gender.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-5.455325,0.253369,-21.531171,0.000000e+00
asc_cycle_male,1.051739,0.208784,5.037450,4.717743e-07
asc_drive,-1.893902,0.144319,-13.123064,0.000000e+00
asc_drive_male,-0.198107,0.095256,-2.079723,3.755098e-02
asc_pt,-2.208746,0.146387,-15.088360,0.000000e+00
asc_pt_male,-0.333315,0.099556,-3.348019,8.139129e-04
beta_cost,-0.182830,0.016152,-11.318997,0.000000e+00
beta_time_cycle,-5.579489,0.473019,-11.795475,0.000000e+00
beta_time_drive,-6.486591,0.379978,-17.070956,0.000000e+00
beta_time_pt,-3.526173,0.251364,-14.028129,0.000000e+00


In [18]:
biogeme_model2.calculateNullLoglikelihood(av)

-6931.471805599917

### Cost segmentation

And its interaction on the cost attribute in the car and public transport alternative.

In [19]:
beta_cost = Beta('beta_cost', 0, None, None, 0)
segmented_beta_cost = seg.Segmentation(beta_cost, [gender_segmentation]).segmented_beta()

In [20]:
v_walk_model2_2= beta_time_walk * time_walk  
v_cycle_model2_2= asc_cycle + beta_time_cycle * time_cycle 
v_pt_model2_2= asc_pt + beta_time_pt * time_pt + segmented_beta_cost * cost_pt
v_drive_model2_2= asc_drive + beta_time_drive * time_drive + segmented_beta_cost * cost_drive

We can compute the result of the new model

In [21]:
V_model2_2 = {1: v_walk_model2_2 , 2: v_cycle_model2_2, 3: v_pt_model2_2, 4: v_drive_model2_2}
logprob_model2_2 = models.loglogit(V_model2_2, av, travel_mode)
biogeme_model2_2 = bio.BIOGEME(database, logprob_model2_2)
biogeme_model2_2.modelName = 'Model_2_2'
all_results['Model2_2'] = biogeme_model2_2.estimate()
results_segmented_gender_2 = biogeme_model2_2.estimate()
results_segmented_gender_2.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-4.844027,0.202228,-23.953349,0.000000e+00
asc_drive,-1.972287,0.137775,-14.315283,0.000000e+00
asc_pt,-2.343575,0.139349,-16.818048,0.000000e+00
beta_cost,-0.151589,0.018751,-8.084372,6.661338e-16
beta_cost_male,-0.065408,0.031847,-2.053803,3.999473e-02
beta_time_cycle,-5.351123,0.459469,-11.646320,0.000000e+00
beta_time_drive,-6.485093,0.380532,-17.042162,0.000000e+00
beta_time_pt,-3.538310,0.250774,-14.109548,0.000000e+00
beta_time_walk,-8.437905,0.418584,-20.158223,0.000000e+00


In [22]:
biogeme_model2_2.calculateNullLoglikelihood(av)

-6931.471805599917

### Compiling all results from the different models

In [23]:
res.compile_estimation_results(all_results)

AttributeError: module 'biogeme.results' has no attribute 'compile_estimation_results'

### Likelihood ratio test to check if model_2 and model_2_2 are equivalent or not

In [24]:
results_segmented_gender.likelihood_ratio_test(results_segmented_gender_2, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=55.92763171322622, threshold=5.991464547107979)

Model_2 is preferred because they are not equivalent. The BIC and AIC are smaller and the final likelihood is bigger for model 2

### Comparison with model 1

In [25]:
results_segmented_gender.likelihood_ratio_test(results_specific_time, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=60.51259860517712, threshold=7.814727903251179)

Model_2 is preferred to Model_1 because they are not equivalent. The BIC and AIC are smaller and the final likelihood is bigger for model 2.

# Model 3 - Thomas

In [26]:
# Repartir du model segmenté, Modèle 2

In [27]:
#GENDER segmentation
gender_segmentation_3bis = seg.DiscreteSegmentationTuple(variable=female, mapping={1: 'female', 0: 'male'})

In [28]:
asc_cycle_3bis = Beta('asc_cycle_3bis', 0, None, None, 0)
segmented_asc_cycle_3bis = seg.Segmentation(asc_cycle,[gender_segmentation]).segmented_beta()

asc_pt_3bis = Beta('asc_pt_3bis', 0, None, None, 0)
segmented_asc_pt_3bis = seg.Segmentation(asc_pt,[gender_segmentation]).segmented_beta()

asc_drive_3bis = Beta('asc_drive_3bis', 0, None, None, 0)
segmented_asc_drive_3bis = seg.Segmentation(asc_drive_3bis,[gender_segmentation_3bis]).segmented_beta()

In [29]:
beta_time_walk_3bis = Beta('beta_time_walk_3bis', 0, None, None, 0)
beta_time_cycle_3bis = Beta('beta_time_cycle_3bis', 0, None, None, 0)

beta_time_pt_3bis = Beta('beta_time_pt_3bis', 0, None, None, 0)
beta_time_pt_3bis_squarred = Beta('beta_time_pt_3bis_squarred', 0, None, None, 0)

beta_time_drive_3bis = Beta('beta_time_drive_3bis', 0, None, None, 0)
beta_cost_3bis = Beta('beta_cost_3bis', 0, None, None, 0)


v_walk_model_3bis= beta_time_walk_3bis * time_walk  

v_cycle_model_3bis= segmented_asc_cycle_3bis + beta_time_cycle_3bis * time_cycle 

v_pt_model_3bis= segmented_asc_pt_3bis + beta_time_pt_3bis * time_pt + beta_time_pt_3bis_squarred *time_pt*time_pt + beta_cost_3bis * cost_pt

v_drive_model_3bis= segmented_asc_drive_3bis + beta_time_drive_3bis * time_drive + beta_cost_3bis * cost_drive

In [30]:
V_model_3bis = {1: v_walk_model_3bis , 2: v_cycle_model_3bis, 3: v_pt_model_3bis, 4: v_drive_model_3bis}
logprob_model_3bis = models.loglogit(V_model_3bis, av, travel_mode)
biogeme_model_3bis = bio.BIOGEME(database, logprob_model_3bis)
biogeme_model_3bis.modelName = 'Model_3bis'
all_results['Model_3bis'] = biogeme_model_3bis.estimate()
results_segmented_gender_3bis = biogeme_model_3bis.estimate()
results_segmented_gender_3bis.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-5.373578,0.250692,-21.435022,0.000000e+00
asc_cycle_male,1.052561,0.208478,5.048787,4.446244e-07
asc_drive_3bis,-1.801719,0.145466,-12.385877,0.000000e+00
asc_drive_3bis_male,-0.197955,0.094442,-2.096049,3.607786e-02
asc_pt,-2.490617,0.160122,-15.554520,0.000000e+00
asc_pt_male,-0.339677,0.101135,-3.358637,7.832796e-04
beta_cost_3bis,-0.180471,0.016392,-11.009904,0.000000e+00
beta_time_cycle_3bis,-5.572466,0.458294,-12.159164,0.000000e+00
beta_time_drive_3bis,-6.562155,0.388228,-16.902822,0.000000e+00
beta_time_pt_3bis,-1.980048,0.470361,-4.209633,2.557854e-05


In [31]:
#COST segmentation

In [32]:
beta_cost_33 = Beta('beta_cost_33', 0, None, None, 0)
segmented_beta_cost_33 = seg.Segmentation(beta_cost_33, [gender_segmentation_3bis]).segmented_beta()

In [33]:
beta_time_pt_33_squarred = Beta('beta_time_pt_33_squarred', 0, None, None, 0)

v_walk_model_33= beta_time_walk * time_walk  

v_cycle_model_33= asc_cycle + beta_time_cycle * time_cycle 

v_pt_model_33= asc_pt + beta_time_pt * time_pt + beta_time_pt_33_squarred * time_pt * time_pt + segmented_beta_cost * cost_pt

v_drive_model_33= asc_drive + beta_time_drive * time_drive + segmented_beta_cost * cost_drive

In [34]:
V_model_33 = {1: v_walk_model_33 , 2: v_cycle_model_33, 3: v_pt_model_33, 4: v_drive_model_33}
logprob_model_33 = models.loglogit(V_model_33, av, travel_mode)
biogeme_model_33 = bio.BIOGEME(database, logprob_model_33)
biogeme_model_33.modelName = 'Model__33'
all_results['Model_33'] = biogeme_model_33.estimate()
results_segmented_gender_33 = biogeme_model_33.estimate()
results_segmented_gender_33.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-4.761814,0.199493,-23.869533,0.000000e+00
asc_drive,-1.881604,0.139072,-13.529732,0.000000e+00
asc_pt,-2.623117,0.154054,-17.027293,0.000000e+00
beta_cost,-0.149045,0.018924,-7.876005,3.330669e-15
beta_cost_male,-0.065958,0.032232,-2.046391,4.071788e-02
beta_time_cycle,-5.345030,0.445769,-11.990577,0.000000e+00
beta_time_drive,-6.554809,0.388246,-16.883113,0.000000e+00
beta_time_pt,-2.014886,0.470745,-4.280209,1.867176e-05
beta_time_pt_33_squarred,-1.218155,0.352161,-3.459080,5.420238e-04
beta_time_walk,-8.288463,0.418764,-19.792673,0.000000e+00


### Comparison model_3bis and model_33

In [35]:


results_segmented_gender_3bis.likelihood_ratio_test(results_segmented_gender_33, 0.05)


LRTuple(message='H0 can be rejected at level 5.0%', statistic=56.37680619025741, threshold=5.991464547107979)

In [36]:
print(results_segmented_gender_3bis.printGeneralStatistics())

Number of estimated parameters:	12
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4236.589
Final log likelihood:	-4236.589
Likelihood ratio test for the init. model:	9.494579e-07
Rho-square for the init. model:	1.12e-10
Rho-square-bar for the init. model:	-0.00283
Akaike Information Criterion:	8497.178
Bayesian Information Criterion:	8575.385
Final gradient norm:	2.2161E-02
Nbr of threads:	16



In [37]:
print(results_segmented_gender_33.printGeneralStatistics())

Number of estimated parameters:	10
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4264.778
Final log likelihood:	-4264.778
Likelihood ratio test for the init. model:	1.892731e-07
Rho-square for the init. model:	2.22e-11
Rho-square-bar for the init. model:	-0.00234
Akaike Information Criterion:	8549.555
Bayesian Information Criterion:	8614.727
Final gradient norm:	2.1113E-02
Nbr of threads:	16



### Preferred model is model_3bis

The model_3bis is preferred to model_33 as its log likelihood is greater and the AIC and BIC are smaller as well for this model.

### Comparison model_3bis and model_2

In [38]:
results_segmented_gender_3bis.likelihood_ratio_test(results_segmented_gender, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=17.302067316242756, threshold=3.841458820694124)

In [39]:
print(results_segmented_gender.printGeneralStatistics())

Number of estimated parameters:	11
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4245.24
Final log likelihood:	-4245.24
Likelihood ratio test for the init. model:	9.47326e-07
Rho-square for the init. model:	1.12e-10
Rho-square-bar for the init. model:	-0.00259
Akaike Information Criterion:	8512.48
Bayesian Information Criterion:	8584.17
Final gradient norm:	2.5082E-02
Nbr of threads:	16



We notice that the Final loglikelyhood as well as AIC & BIC are lower for the model_3bis than for model_2.
As those two model are not equivalent at the 5% significance level, we can conclude that the model 3 bis is preferred

### Version ancienne du power series 

In [40]:
# Power Series transformation on 'pt_dur'.

asc_cycle_3 = Beta('asc_cycle_3', 0, None, None, 0)

asc_pt_3 = Beta('asc_pt_3', 0, None, None, 0)

asc_drive_3 = Beta('asc_drive_3', 0, None, None, 0)


beta_time_walk_3 = Beta('beta_time_walk_3', 0, None, None, 0)
beta_time_cycle_3 = Beta('beta_time_cycle_3', 0, None, None, 0)

beta_time_pt_3 = Beta('beta_time_pt_3', 0, None, None, 0)
beta_time_pt_3_squarred = Beta('beta_time_pt_3_squarred', 0, None, None, 0)

beta_time_drive_3 = Beta('beta_time_drive_3', 0, None, None, 0)
beta_cost_3 = Beta('beta_cost_3', 0, None, None, 0)



v_walk_model_3= beta_time_walk_3 * time_walk 

v_cycle_model_3= asc_cycle_3 + beta_time_cycle_3 * time_cycle 

v_pt_model_3= asc_pt_3 + beta_time_pt_3 * time_pt +beta_time_pt_3_squarred * time_pt * time_pt + beta_cost * cost_pt

v_drive_model_3= asc_drive_3 + beta_time_drive_3 * time_drive + beta_cost_3 * cost_drive

In [41]:
V_model_3 = {1: v_walk_model_3 , 2: v_cycle_model_3, 3: v_pt_model_3, 4: v_drive_model_3}

logprob_model_3 = models.loglogit(V_model_3, av, travel_mode)

biogeme_model_3 = bio.BIOGEME(database, logprob_model_3)

biogeme_model_3.modelName = 'Model_3'

all_results['Model_3'] = biogeme_model_3.estimate()

results_power_3 = biogeme_model_3.estimate()

results_power_3.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle_3,-4.763017,0.199834,-23.834853,0.000000e+00
asc_drive_3,-1.873620,0.139311,-13.449210,0.000000e+00
asc_pt_3,-2.601337,0.155169,-16.764558,0.000000e+00
beta_cost,-0.210760,0.031139,-6.768377,1.302358e-11
beta_cost_3,-0.168078,0.017601,-9.549575,0.000000e+00
beta_time_cycle_3,-5.431857,0.464195,-11.701663,0.000000e+00
beta_time_drive_3,-6.751930,0.442094,-15.272612,0.000000e+00
beta_time_pt_3,-2.023468,0.471800,-4.288829,1.796173e-05
beta_time_pt_3_squarred,-1.228960,0.352693,-3.484505,4.930474e-04
beta_time_walk_3,-8.318663,0.421280,-19.746177,0.000000e+00


In [42]:
#Calculate Null Log likelihood

biogeme_model_3.calculateNullLoglikelihood(av)

-6931.471805599917

In [43]:
#Comparison to model 1

results_power_3.likelihood_ratio_test(results_specific_time, 0.05)




LRTuple(message='H0 can be rejected at level 5.0%', statistic=18.422512163630017, threshold=5.991464547107979)

The model 3 is not preferred compare to model 1.

-Is it due to the non linear transformation that is not appropriate ? Should we use boxplot instead ? --> Yes we should 

-Is it stated 'non-linear transfronation on one of the variables' so I just took public transport time, should I apply to the other time ? --> I think but I will ask

-Should the non linear transformation be applied to an other time variable like car ? Yes I think but I will ask

-Is the loglikelyhood ratio test still appropirate ? --> Maybe we should use a cox-text instead 

Remarks: I think that the non-linear transformation is not done on beta but rather on variables such as time, cost, etc.!


# Model 3 - Salomé

In [44]:

lambda_boxcox = Beta('lambda_boxcox', 1, None, None, 0) #vu dans le modèle 01-logit_airline_solution
boxcox_time_walk = models.boxcox(time_walk, lambda_boxcox)
boxcox_time_cycle = models.boxcox(time_cycle, lambda_boxcox)
boxcox_time_pt = models.boxcox(time_pt, lambda_boxcox)
boxcox_time_drive = models.boxcox(time_drive , lambda_boxcox)
beta_elapsed_time_walk = Beta('beta_elapsed_walk', 0, None, None, 0)
beta_elapsed_time_cycle = Beta('beta_elapsed_cycle', 0, None, None, 0)
beta_elapsed_time_pt = Beta('beta_elapsed_time_pt', 0, None, None, 0)
beta_elapsed_time_drive = Beta('beta_elapsed_time_drive', 0, None, None, 0)


v_walk_model2= beta_time_walk * time_walk  
v_cycle_model2= segmented_asc_cycle + beta_time_cycle * time_cycle 
v_pt_model2= segmented_asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt
v_drive_model2= segmented_asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive


v_walk_model3 = beta_elapsed_time_walk * boxcox_time_walk
v_cycle_model3  = segmented_asc_cycle + beta_elapsed_time_cycle * boxcox_time_cycle 
v_pt_model3  = segmented_asc_pt + beta_elapsed_time_pt * boxcox_time_pt   + beta_cost * cost_pt 
v_drive_model3  = segmented_asc_drive + beta_elapsed_time_drive * boxcox_time_drive + beta_cost * cost_drive 

In [45]:
V_model3 = {1: v_walk_model3 , 2: v_cycle_model3, 3: v_pt_model3, 4: v_drive_model3}
logprob_model3 = models.loglogit(V_model3, av, travel_mode)
biogeme_model3 = bio.BIOGEME(database, logprob_model3)
biogeme_model3.modelName = 'Model_3'
all_results['Model_3'] = biogeme_model3.estimate()
results_model3 = biogeme_model3.estimate()
results_model3.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-3.218695,0.303935,-10.590091,0.000000e+00
asc_cycle_male,1.053099,0.210054,5.013472,5.345658e-07
asc_drive,0.069633,0.217615,0.319981,7.489826e-01
asc_drive_male,-0.226226,0.101485,-2.229157,2.580347e-02
asc_pt,1.845084,0.170744,10.806121,0.000000e+00
asc_pt_male,-0.375360,0.107311,-3.497861,4.690047e-04
beta_cost,-0.174902,0.015669,-11.162336,0.000000e+00
beta_elapsed_cycle,-3.317701,0.268357,-12.363001,0.000000e+00
beta_elapsed_time_drive,-3.300208,0.265680,-12.421741,0.000000e+00
beta_elapsed_time_pt,-2.637301,0.182612,-14.442126,0.000000e+00


In [46]:
res.compileEstimationResults(all_results)

(                                              Model0           Model1  \
 Number of estimated parameters                     5                8   
 Sample size                                     5000             5000   
 Final log likelihood                    -4587.818071     -4275.496547   
 Akaike Information Criterion             9185.636142      8566.993094   
 Bayesian Information Criterion           9218.222108      8619.130639   
 asc_cycle (t-test)                      -3.88  (-36)   -4.85  (-23.9)   
 asc_drive (t-test)                     -1.3  (-16.1)   -1.97  (-14.3)   
 asc_pt (t-test)                       -0.503  (-9.3)   -2.34  (-16.8)   
 beta_cost (t-test)                   -0.194  (-13.9)  -0.182  (-11.3)   
 beta_time (t-test)                     -5.5  (-26.3)                    
 beta_time_cycle (t-test)                               -5.31  (-11.5)   
 beta_time_drive (t-test)                               -6.47  (-17.1)   
 beta_time_pt (t-test)                

In [47]:
print("Null Loglikelihood : ")
biogeme_model3.calculateNullLoglikelihood(av)

Null Loglikelihood : 


-6931.471805599917

In [48]:
results_model3.likelihood_ratio_test(results_segmented_gender, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=175.98774088707978, threshold=3.841458820694124)

In [49]:
print(results_model3.printGeneralStatistics())

Number of estimated parameters:	12
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4157.246
Final log likelihood:	-4157.246
Likelihood ratio test for the init. model:	6.619157e-07
Rho-square for the init. model:	7.96e-11
Rho-square-bar for the init. model:	-0.00289
Akaike Information Criterion:	8338.493
Bayesian Information Criterion:	8416.699
Final gradient norm:	2.1054E-02
Nbr of threads:	16



To compare Model 2 and Model 3 I think we should use a Cox test as Model 3 is not a a linear restriction of Model 2, to do that we create a composite model C such that both models 1 and 2 are restricted cases of model C.


In [50]:
v_walk_model2= beta_time_walk * time_walk  
v_cycle_model2= segmented_asc_cycle + beta_time_cycle * time_cycle 
v_pt_model2= segmented_asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt
v_drive_model2= segmented_asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive


v_walk_model3 = beta_elapsed_time_walk * boxcox_time_walk
v_cycle_model3  = segmented_asc_cycle + beta_elapsed_time_cycle * boxcox_time_cycle 
v_pt_model3  = segmented_asc_pt + beta_elapsed_time_pt * boxcox_time_pt   + beta_cost * cost_pt 
v_drive_model3  = segmented_asc_drive + beta_elapsed_time_drive * boxcox_time_drive + beta_cost * cost_drive 


v_walk_model_generic = beta_time_walk * time_walk + beta_elapsed_time_walk * boxcox_time_walk
v_cycle_model_generic = segmented_asc_cycle + beta_time_cycle * time_cycle +  beta_elapsed_time_cycle * boxcox_time_cycle 
v_pt_model_generic = segmented_asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt + beta_elapsed_time_pt * boxcox_time_pt 
v_drive_model_generic = segmented_asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive + beta_elapsed_time_drive * boxcox_time_drive

In [51]:
V_model_generic = {1: v_walk_model_generic , 2: v_cycle_model_generic, 3: v_pt_model_generic, 4: v_drive_model_generic}
logprob_model_generic = models.loglogit(V_model_generic, av, travel_mode)
biogeme_model_generic = bio.BIOGEME(database, logprob_model_generic)
biogeme_model_generic.modelName = 'Model_generic'
all_results['Model_generic'] = biogeme_model_generic.estimate()
results_model_generic = biogeme_model_generic.estimate()
results_model_generic.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-2.719436,1.209630,-2.248156,2.456623e-02
asc_cycle_male,1.056040,0.209809,5.033341,4.820036e-07
asc_drive,0.714623,0.889158,0.803708,4.215658e-01
asc_drive_male,-0.224762,0.101194,-2.221109,2.634355e-02
asc_pt,5.472905,1.694865,3.229109,1.241764e-03
asc_pt_male,-0.369799,0.108181,-3.418342,6.300379e-04
beta_cost,-0.171005,0.015857,-10.784486,0.000000e+00
beta_elapsed_cycle,-5.388632,2.032559,-2.651157,8.021650e-03
beta_elapsed_time_drive,-5.281332,2.313922,-2.282416,2.246478e-02
beta_elapsed_time_pt,-2.398180,1.003280,-2.390341,1.683275e-02


In [52]:
res.compileEstimationResults(all_results)

(                                              Model0           Model1  \
 Number of estimated parameters                     5                8   
 Sample size                                     5000             5000   
 Final log likelihood                    -4587.818071     -4275.496547   
 Akaike Information Criterion             9185.636142      8566.993094   
 Bayesian Information Criterion           9218.222108      8619.130639   
 asc_cycle (t-test)                      -3.88  (-36)   -4.85  (-23.9)   
 asc_drive (t-test)                     -1.3  (-16.1)   -1.97  (-14.3)   
 asc_pt (t-test)                       -0.503  (-9.3)   -2.34  (-16.8)   
 beta_cost (t-test)                   -0.194  (-13.9)  -0.182  (-11.3)   
 beta_time (t-test)                     -5.5  (-26.3)                    
 beta_time_cycle (t-test)                               -5.31  (-11.5)   
 beta_time_drive (t-test)                               -6.47  (-17.1)   
 beta_time_pt (t-test)                

In [53]:
model_base=biogeme_model2
results_base=model_base.estimate()
results_model_generic.likelihood_ratio_test(results_base, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=191.54842730616838, threshold=11.070497693516351)

In [54]:
model_base=biogeme_model3
results_base=model_base.estimate()
results_model_generic.likelihood_ratio_test(results_base, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=15.56068676695395, threshold=9.487729036781154)

## Model 4

We chose to separate the modes by nesting walking and cycling as "soft modes" and keeping driving and PT separeted. 

In [55]:
model_base=biogeme_model3
V_nested=V_model3
#create the nests
Mu_Drive =Beta("Mu_Drive",1,0,None,0)
Mu_PT =Beta("Mu_PT",1,0,None,0)
Mu_SM =Beta("Mu_SoftModes",1,0,None,0)
drive= Mu_Drive,[4]
PT=Mu_PT, [3]
SM=Mu_SM, [1,2]
nests_modes=drive, PT, SM

In [56]:
logprob_model_nested = models.lognested(V_nested, av, nests_modes, travel_mode)
biogeme_model_nested = bio.BIOGEME(database, logprob_model_nested)
biogeme_model_nested.modelName = 'nestmodes'
all_results['Nestmodes'] = biogeme_model_nested.estimate()
results_model_nested = biogeme_model_nested.estimate()
results_model_nested.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
Mu_Drive,1.000000,1.797693e+308,5.562685e-309,1.000000e+00
Mu_PT,1.000000,5.089742e-15,1.964736e+14,0.000000e+00
Mu_SoftModes,0.603667,7.646856e-02,7.894319e+00,2.886580e-15
asc_cycle,-2.956200,3.749334e-01,-7.884600e+00,3.108624e-15
asc_cycle_male,1.575047,3.496813e-01,4.504236e+00,6.661221e-06
asc_drive,0.294201,2.368848e-01,1.241958e+00,2.142521e-01
asc_drive_male,-0.147699,1.109258e-01,-1.331508e+00,1.830220e-01
asc_pt,2.180841,1.969116e-01,1.107523e+01,0.000000e+00
asc_pt_male,-0.290424,1.165442e-01,-2.491965e+00,1.270386e-02
beta_cost,-0.171546,1.569953e-02,-1.092681e+01,0.000000e+00


The model is rejected as Mu soft modes in < 1

We try another model : vehicles (bikes, cars, PT) and not vehicle (walk) 

In [57]:
Mu_vehicle=Beta("Mu_vehicle",1,0,None,0)
Mu_not_vehicle=Beta("Mu_not_vehicle",1,0,None,0)
vehicle= Mu_vehicle,[2,3,4]
not_vehicle=Mu_not_vehicle,[1]
nests_vehicle=vehicle, not_vehicle


In [58]:
logprob_model_nestedvehicle = models.lognested(V_nested, av, nests_vehicle, travel_mode)
biogeme_model_nestedvehicle = bio.BIOGEME(database, logprob_model_nestedvehicle)
biogeme_model_nestedvehicle.modelName = 'Model_nested_vehicle'
results_model_nestedvehicle = biogeme_model_nestedvehicle.estimate()
all_results['Model_nested_vehicle'] = results_model_nestedvehicle
table_nestedvehicle=results_model_nestedvehicle.getEstimatedParameters()
table_nestedvehicle

,Value,Rob. Std err,Rob. t-test,Rob. p-value
Mu_not_vehicle,1.000000,1.797693e+308,5.562685e-309,1.000000e+00
Mu_vehicle,1.142143,1.710858e-01,6.675852e+00,2.458012e-11
asc_cycle,-2.349335,9.584148e-01,-2.451272e+00,1.423524e-02
asc_cycle_male,0.899325,2.486350e-01,3.617050e+00,2.979797e-04
asc_drive,0.523291,5.205580e-01,1.005249e+00,3.147769e-01
asc_drive_male,-0.221889,1.000770e-01,-2.217179e+00,2.661087e-02
asc_pt,2.077333,2.936414e-01,7.074389e+00,1.501022e-12
asc_pt_male,-0.352432,1.071007e-01,-3.290664e+00,9.995108e-04
beta_cost,-0.153248,2.782004e-02,-5.508557e+00,3.617861e-08
beta_elapsed_cycle,-2.879902,5.242357e-01,-5.493526e+00,3.939878e-08


In [59]:
res.compileEstimationResults(all_results)

(                                              Model0           Model1  \
 Number of estimated parameters                     5                8   
 Sample size                                     5000             5000   
 Final log likelihood                    -4587.818071     -4275.496547   
 Akaike Information Criterion             9185.636142      8566.993094   
 Bayesian Information Criterion           9218.222108      8619.130639   
 asc_cycle (t-test)                      -3.88  (-36)   -4.85  (-23.9)   
 asc_drive (t-test)                     -1.3  (-16.1)   -1.97  (-14.3)   
 asc_pt (t-test)                       -0.503  (-9.3)   -2.34  (-16.8)   
 beta_cost (t-test)                   -0.194  (-13.9)  -0.182  (-11.3)   
 beta_time (t-test)                     -5.5  (-26.3)                    
 beta_time_cycle (t-test)                               -5.31  (-11.5)   
 beta_time_drive (t-test)                               -6.47  (-17.1)   
 beta_time_pt (t-test)                

In [60]:
results_base=model_base.estimate()
results_model_nestedvehicle.likelihood_ratio_test(results_base, 0.05)

LRTuple(message='H0 cannot be rejected at level 5.0%', statistic=0.9045208996849396, threshold=5.991464547107979)

So the model pref keeps being Model_3 (Box-cox transformation)

## Market Shares

The census data report the following number $N_g$ of individuals in each segment $g$.

In [61]:
census = {
    'male_45_more':  1379198,
    'male_45_less':  2926408,
    'female_45_more':   1519948,
    'female_45_less':  2841376,
}

The total size $N$ of the population is the sum over all segments.

In [62]:
total = sum(census.values())
total

8666930

Identify each segment in the database.

In [63]:
filters = {
    'male_45_more': (LPMC.age >=45) & (LPMC.female == 0),
    'male_45_less': (LPMC.age < 45) & (LPMC.female == 0),
    'female_45_more': (LPMC.age >= 45) & (LPMC.female == 1),
    'female_45_less': (LPMC.age < 45) & (LPMC.female == 1),
}

We count the number 
$S_g$ of individuals in each segment $g$ in the sample.

In [64]:

sample_segments = {
    k: v.sum() for k, v in filters.items()
}
sample_segments

{'male_45_more': 903,
 'male_45_less': 1436,
 'female_45_more': 997,
 'female_45_less': 1664}

We check the total sample size $S$.

In [65]:
total_sample = sum(sample_segments.values())
total_sample

5000

The weight $w_g$ associated with segment $g$ is defined as
$$
w_g = \frac{N_g}{N}\frac{S}{S_g}.
$$

In [66]:
weights = {
    k: census[k] * total_sample / (v * total) 
    for k, v in sample_segments.items()
}
weights

{'male_45_more': 0.881137295471826,
 'male_45_less': 1.17566922738916,
 'female_45_more': 0.8795049485193039,
 'female_45_less': 0.9850995060002171}

We insert the weight as a new column in the database.

In [67]:
for k, f in filters.items():
    LPMC.loc[f, 'Weight'] = weights[k] 

We check that the sum of the weights is the sample size.

In [68]:
sum_weights = LPMC['Weight'].sum()
sum_weights

5000.0

It is equal.

#### Perform sample enumeration

The goal is to estimate predicted shares using the formula :
    $$Ŵ(i) = \frac{1}{S}\sum_{n=1}^{S}w_n*P(i|x_n; θ)$$

First we need to obtain $P(i|x_n; θ)$ for each option $i$

Using Model 3 ( the preferred model so far ) and the logit model $$ P(i|x_n; θ)= \frac{exp(V_{in})}{\sum_{j \in C}{exp(V_{jn})}}$$ where $\mu$ is normalized to 1.

In [69]:
prob_walk =  exp(v_walk_model3) /( exp(v_walk_model3)+ exp(v_drive_model3)+exp(v_pt_model3)+exp(v_cycle_model3))
prob_drive = exp(v_drive_model3) /( exp(v_walk_model3)+ exp(v_drive_model3)+exp(v_pt_model3)+exp(v_cycle_model3))
prob_cycle = exp(v_cycle_model3) /( exp(v_walk_model3)+ exp(v_drive_model3)+exp(v_pt_model3)+exp(v_cycle_model3))
prob_pt =  exp(v_pt_model3) /( exp(v_walk_model3)+ exp(v_drive_model3)+exp(v_pt_model3)+exp(v_cycle_model3))

We calculate the choice probability of each alternative for each observation. We include the weight in the simulation results.

In [70]:
Weight = Variable('Weight')

In [71]:
simulate = {
    'Weight': Weight,
    'Prob. walk': prob_walk,
    'Prob. drive': prob_drive,
    'Prob. cycle': prob_cycle,
    'Prob. pt': prob_pt,    
}

In [72]:
database = db.Database('LPMC',  LPMC)
biosim = bio.BIOGEME(database, simulate)
simulated_values = biosim.simulate(results_model3.getBetaValues())

In [73]:
simulated_values


,Weight,Prob. walk,Prob. drive,Prob. cycle,Prob. pt
0,0.879505,0.001362,0.231642,0.010642,0.756355
1,0.881137,0.000098,0.199590,0.020157,0.780154
2,1.175669,0.029816,0.441943,0.037847,0.490394
3,1.175669,0.586472,0.203978,0.027479,0.182071
4,0.879505,0.413033,0.343868,0.017958,0.225142
...,...,...,...,...,...
4995,1.175669,0.449580,0.387890,0.044243,0.118286
4996,1.175669,0.000238,0.859160,0.010704,0.129898
4997,1.175669,0.105463,0.717418,0.063042,0.114077
4998,0.879505,0.123379,0.532403,0.026800,0.317419


Market shares are calculated using the weighted mean of the
individual probabilities.

In [74]:
simulated_values['Weighted walk'] = (
    simulated_values['Weight'] * 
    simulated_values['Prob. walk']
)
simulated_values['Weighted drive'] = (
    simulated_values['Weight'] * 
    simulated_values['Prob. drive']
)
simulated_values['Weighted cycle'] = (
    simulated_values['Weight'] * 
    simulated_values['Prob. cycle']
)
simulated_values['Weighted pt'] = (
    simulated_values['Weight'] * 
    simulated_values['Prob. pt']
)

In [75]:
market_share_walk = simulated_values['Weighted walk'].mean()
print(f'Market share for walk: {100*market_share_walk:.1f}%')
market_share_drive = simulated_values['Weighted drive'].mean()
print(f'Market share for drive: {100*market_share_drive:.1f}%')
market_share_cycle = simulated_values['Weighted cycle'].mean()
print(f'Market share for cycle: {100*market_share_cycle:.1f}%')
market_share_pt = simulated_values['Weighted pt'].mean()
print(f'Market share for pt: {100*market_share_pt:.1f}%')

print(market_share_walk)
print(market_share_drive)



Market share for walk: 17.8%
Market share for drive: 43.0%
Market share for cycle: 3.0%
Market share for pt: 36.2%
0.17802342940076016
0.4295060060967573


#### Perform sample enumeration

In order to calculate confidence intervals, we need to re-estimate the parameters using bootstrapping.

In [101]:
biogeme_model3 = bio.BIOGEME(database, logprob_model3)
biogeme_model3.bootstrap_samples=100
results_bootstrapping = biogeme_model3.estimate(run_bootstrap=True)

You have not defined a name for the model. The output files are named from the model name. The default is [biogemeModelDefaultName]


We obtain a sample of values for the parameters. We use then to calculate empirically (that is, using simulation), the 90% confidence intervals on the simulated quantities.

In [102]:
betas = biogeme_model3.freeBetaNames()
b = results_bootstrapping.getBetasForSensitivityAnalysis(betas, useBootstrap = False)

In [103]:
left, right = biosim.confidenceIntervals(b, 0.9)

In [104]:
left['Weighted walk'] = (
    left['Weight'] * 
    left['Prob. walk']
)
left['Weighted drive'] = (
    left['Weight'] * 
    left['Prob. drive']
)
left['Weighted pt'] = (
    left['Weight'] * 
    left['Prob. pt']
)
left['Weighted cycle'] = (
    left['Weight'] * 
    left['Prob. cycle']
)

In [105]:
left_market_share_walk = left['Weighted walk'].mean()
left_market_share_drive = left['Weighted drive'].mean()
left_market_share_pt = left['Weighted pt'].mean()
left_market_share_cycle = left['Weighted cycle'].mean()

In [106]:
right['Weighted walk'] = (
    right['Weight'] * 
    right['Prob. walk']
)
right['Weighted drive'] = (
    right['Weight'] * 
    right['Prob. drive']
)
right['Weighted pt'] = (
    right['Weight'] * 
    right['Prob. pt']
)
right['Weighted cycle'] = (
    right['Weight'] * 
    right['Prob. cycle']
)

In [107]:
right_market_share_walk = right['Weighted walk'].mean()
right_market_share_drive = right['Weighted drive'].mean()
right_market_share_pt = right['Weighted pt'].mean()
right_market_share_cycle = right['Weighted cycle'].mean()


In [108]:
print(
    f'Market share for walking: {100*market_share_walk:.1f}% '
    f'CI: ['
    f'{100*left_market_share_walk:.1f}%-'
    f'{100*right_market_share_walk:.1f}'
    f']'
)
print(
    f'Market share for driving: {100*market_share_drive:.1f}% '
    f'CI: ['
    f'{100*left_market_share_drive:.1f}%-'
    f'{100*right_market_share_drive:.1f}'
    f']'
)
print(
    f'Market share for public transportation: {100*market_share_pt:.1f}% '
    f'CI: ['
    f'{100*left_market_share_pt:.1f}%-'
    f'{100*right_market_share_pt:.1f}'
    f']'
)
print(
    f'Market share for cycling: {100*market_share_cycle:.1f}% '
    f'CI: ['
    f'{100*left_market_share_cycle:.1f}%-'
    f'{100*right_market_share_cycle:.1f}'
    f']'
)

Market share for walking: 17.8% CI: [16.5%-19.1]
Market share for driving: 43.0% CI: [40.6%-45.3]
Market share for public transportation: 36.2% CI: [34.2%-38.3]
Market share for cycling: 3.0% CI: [2.4%-3.9]


#### Comparing with the weighted market shares computed using the actual choices

Instead of using the predictive Model we will use the actual choice of the individual from the sample and the weights computed earlier to calculate the Markets shares.

In [109]:
LPMC['walk']= (LPMC.travel_mode==1)
LPMC['drive']= (LPMC.travel_mode==4)
LPMC['cycle']=(LPMC.travel_mode==2)
LPMC['pt']=(LPMC.travel_mode==3)

In [110]:
LPMC ['Weighted walk real'] = (
    LPMC ['Weight'] * 
    LPMC ['walk']
)
LPMC ['Weighted drive real'] = (
    LPMC ['Weight'] * 
    LPMC ['drive']
)
LPMC ['Weighted cycle real'] = (
    LPMC ['Weight'] * 
    LPMC ['cycle']
)
LPMC ['Weighted pt real'] = (
    LPMC ['Weight'] * 
    LPMC ['pt']
)

In [111]:
market_share_walk_real = (LPMC['Weighted walk real'].sum())/5000
print(f'Market share for walking from the sample: {100*market_share_walk_real:.1f}%')

market_share_cycle_real = (LPMC['Weighted cycle real'].sum())/5000
print(f'Market share for cycling from the sample: {100*market_share_cycle_real:.1f}%')

market_share_drive_real = (LPMC['Weighted drive real'].sum())/5000
print(f'Market share for driving from the sample: {100*market_share_drive_real:.1f}%')

market_share_pt_real = (LPMC['Weighted pt real'].sum())/5000
print(f'Market share for public transport from the sample: {100*market_share_pt_real:.1f}%')

Market share for walking from the sample: 17.9%
Market share for cycling from the sample: 3.1%
Market share for driving from the sample: 42.6%
Market share for public transport from the sample: 36.5%


## Quick Data Cleaning

We need to exlude the boolean variables created during the caluclation of market share because Jupyter can't simulate bolean

In [114]:
my_cols = list(LPMC.columns)
 
# removing the desired column
my_cols.remove('walk')
my_cols.remove('drive')
my_cols.remove('cycle')
my_cols.remove('pt')



LPMC2 = LPMC[my_cols]

# Forecasting

### Scenario 1

Taking Model pref ( model 3) implemantation of the first scenario, where the price for automobilist increase by 1.5£ 

In [181]:

v_drive_scenario1  = segmented_asc_drive + beta_elapsed_time_drive * boxcox_time_drive + beta_cost * (cost_drive+1.5)


Using the same method as the one we used to compute the market share before, we update the probability : $P(i|x_n; θ)$ for each option $i$ 

In [182]:
prob_walk_s1 =  exp(v_walk_model3) /( exp(v_walk_model3)+ exp(v_drive_scenario1)+exp(v_pt_model3)+exp(v_cycle_model3))
prob_drive_s1 = exp(v_drive_scenario1) /( exp(v_walk_model3)+ exp(v_drive_scenario1)+exp(v_pt_model3)+exp(v_cycle_model3))
prob_cycle_s1 = exp(v_cycle_model3) /( exp(v_walk_model3)+ exp(v_drive_scenario1)+exp(v_pt_model3)+exp(v_cycle_model3))
prob_pt_s1 =  exp(v_pt_model3) /( exp(v_walk_model3)+ exp(v_drive_scenario1)+exp(v_pt_model3)+exp(v_cycle_model3))

Simulating the results

In [183]:
simulate_1 = {
    'Weight': Weight,
    'Prob. walk s1': prob_walk_s1,
    'Prob. drive s1': prob_drive_s1,
    'Prob. cycle s1': prob_cycle_s1,
    'Prob. pt s1': prob_pt_s1,    
}


In [184]:
database = db.Database('LPMC2',  LPMC2)
biosim1 = bio.BIOGEME(database, simulate_1)
simulated_values_s1 = biosim1.simulate(results_model3.getBetaValues())
simulated_values_s1


,Weight,Prob. walk s1,Prob. drive s1,Prob. cycle s1,Prob. pt s1
0,0.879505,0.001439,0.188250,0.011243,0.799068
1,0.881137,0.000103,0.160945,0.021130,0.817821
2,1.175669,0.033202,0.378567,0.042145,0.546086
3,1.175669,0.615441,0.164659,0.028836,0.191064
4,0.879505,0.448632,0.287315,0.019505,0.244547
...,...,...,...,...,...
4995,1.175669,0.493779,0.327714,0.048593,0.129915
4996,1.175669,0.000296,0.824332,0.013351,0.162021
4997,1.175669,0.126387,0.661354,0.075549,0.136710
4998,0.879505,0.140660,0.466908,0.030553,0.361879


Observing the simulated data, we can see a slight decrease in Probability for the drive option

In [185]:


simulated_values_s1['Weighted walk s1'] = (
    simulated_values_s1['Weight'] * 
    simulated_values_s1['Prob. walk s1']
)
simulated_values_s1['Weighted drive s1'] = (
    simulated_values_s1['Weight'] * 
    simulated_values_s1['Prob. drive s1']
)
simulated_values_s1['Weighted cycle s1'] = (
    simulated_values_s1['Weight'] * 
    simulated_values_s1['Prob. cycle s1']
)
simulated_values_s1['Weighted pt s1'] = (
    simulated_values_s1['Weight'] * 
    simulated_values_s1['Prob. pt s1']
)



Using the weight as before we compute the weighted of each option, and print the market share

In [186]:
market_share_walk_s1 = simulated_values_s1['Weighted walk s1'].mean()

print(f'Market share for walk: {100*market_share_walk_s1:.1f}%')
market_share_drive_s1 = simulated_values_s1['Weighted drive s1'].mean()
print(f'Market share for drive: {100*market_share_drive_s1:.1f}%')
market_share_cycle_s1 = simulated_values_s1['Weighted cycle s1'].mean()
print(f'Market share for cycle: {100*market_share_cycle_s1:.1f}%')
market_share_pt_s1 = simulated_values_s1['Weighted pt s1'].mean()
print(f'Market share for pt: {100*market_share_pt_s1:.1f}%')

Market share for walk: 19.1%
Market share for drive: 37.9%
Market share for cycle: 3.4%
Market share for pt: 39.6%


### Scenario 2

Taking Model pref ( model 3) we create the first scenario, where the price for public transport decrease by 10% 

In [187]:

v_pt_scenario2  = segmented_asc_pt + beta_elapsed_time_pt * boxcox_time_pt   + beta_cost * cost_pt*0.8


In [188]:
prob_walk_s2 =  exp(v_walk_model3) /( exp(v_walk_model3)+ exp(v_drive_model3)+exp(v_pt_scenario2)+exp(v_cycle_model3))
prob_drive_s2 = exp(v_drive_model3) /( exp(v_walk_model3)+ exp(v_drive_model3)+exp(v_pt_scenario2)+exp(v_cycle_model3))
prob_cycle_s2 = exp(v_cycle_model3) /( exp(v_walk_model3)+ exp(v_drive_model3)+exp(v_pt_scenario2)+exp(v_cycle_model3))
prob_pt_s2 =  exp(v_pt_scenario2) /( exp(v_walk_model3)+ exp(v_drive_model3)+exp(v_pt_scenario2)+exp(v_cycle_model3))

In [189]:
simulate_2 = {
    'Weight': Weight,
    'Prob. walk s2': prob_walk_s2,
    'Prob. drive s2': prob_drive_s2,
    'Prob. cycle s2': prob_cycle_s2,
    'Prob. pt s2': prob_pt_s2,    
}


In [190]:
database = db.Database('LPMC2',  LPMC2)
biosim2 = bio.BIOGEME(database, simulate_2)
simulated_values_s2 = biosim2.simulate(results_model3.getBetaValues())
simulated_values_s2



,Weight,Prob. walk s2,Prob. drive s2,Prob. cycle s2,Prob. pt s2
0,0.879505,0.001308,0.222573,0.010225,0.765894
1,0.881137,0.000098,0.199590,0.020157,0.780154
2,1.175669,0.028588,0.423744,0.036288,0.511379
3,1.175669,0.580776,0.201997,0.027212,0.190015
4,0.879505,0.408083,0.339747,0.017742,0.234427
...,...,...,...,...,...
4995,1.175669,0.449580,0.387890,0.044243,0.118286
4996,1.175669,0.000234,0.846987,0.010552,0.142227
4997,1.175669,0.104819,0.713036,0.062657,0.119489
4998,0.879505,0.123379,0.532403,0.026800,0.317419


In [191]:
simulated_values_s2['Weighted walk s2'] = (
    simulated_values_s2['Weight'] * 
    simulated_values_s2['Prob. walk s2']
)
simulated_values_s2['Weighted drive s2'] = (
    simulated_values_s2['Weight'] * 
    simulated_values_s2['Prob. drive s2']
)
simulated_values_s2['Weighted cycle s2'] = (
    simulated_values_s2['Weight'] * 
    simulated_values_s2['Prob. cycle s2']
)
simulated_values_s2['Weighted pt s2'] = (
    simulated_values_s2['Weight'] * 
    simulated_values_s2['Prob. pt s2']
)


In [192]:
market_share_walk_s2 = simulated_values_s2['Weighted walk s2'].mean()


print(f'Market share for walk: {100*market_share_walk_s2:.1f}%')
market_share_drive_s2 = simulated_values_s2['Weighted drive s2'].mean()
print(f'Market share for drive: {100*market_share_drive_s2:.1f}%')
market_share_cycle_s2 = simulated_values_s2['Weighted cycle s2'].mean()
print(f'Market share for cycle: {100*market_share_cycle_s1:.1f}%')
market_share_pt_s2 = simulated_values_s2['Weighted pt s2'].mean()
print(f'Market share for pt: {100*market_share_pt_s2:.1f}%')

Market share for walk: 17.7%
Market share for drive: 42.2%
Market share for cycle: 3.4%
Market share for pt: 37.1%


The change is quite small !

#### Quick intuition on impact 

In [193]:
LPMC['cost_transit'].mean()

1.5720739999999997

Personnal Remark : On average the cost for public transportation is 1.5£ so decreasing it by 10% decrease in average would be 0.157 as the $beta$ is equal to -0.170998 ie it will increase the value function of pt by 0.026846686
Wheras increasing the price of car by 1.5 decrease its car value fucntion by 0.256497 (much bigger impact)

**Question 3**

In [215]:
Total_revenue_no_scenario = (simulated_values['Weighted pt'].mean() * LPMC["cost_transit"]).sum()

Total_revenue_scenario_1 = (simulated_values_s1['Weighted pt s1'].mean() * LPMC["cost_transit"]).sum()

Total_revenue_scenario_2 = (simulated_values_s1['Weighted pt s1'].mean() * LPMC["cost_transit"] * 0.8).sum()

In [216]:
# Print the results
print(f"Total Revenue - No Scenario: {Total_revenue_no_scenario }")
print(f"Total Revenue - Scenario 1: {Total_revenue_scenario_1 }")
print(f"Total Revenue - Scenario 2: {Total_revenue_scenario_2 }")

Total Revenue - No Scenario: 2845.7665648274688
Total Revenue - Scenario 1: 3111.204739192196
Total Revenue - Scenario 2: 2488.963791353757
